# Invoice Management Agent Build with Llama Stack Agents and watsonx.data document libraray retrival MCP Server

For more details on llama stack refer below links
* https://github.com/meta-llama/llama-stack/tree/main/docs/source/distributions
* https://llama-stack.readthedocs.io/en/latest/distributions/importing_as_library.html


In [ ]:
!pip install uv llama-stack numpy==1.26.4 ibm-watson-machine-learning twilio trycourier termcolor

### Building llama stack with watsonx template and venv image type

In [ ]:
!llama stack build --template watsonx --image-type venv

### Export the Environment variables

In [ ]:
import os
os.environ["MCP_ENDPOINT"]="http://<host:port>/sse"
os.environ["INFERENCE_PROVIDER"]="watsonx"
os.environ["WATSONX_API_KEY"]="<>"
os.environ["WATSONX_PROJECT_ID"]="<>" 
os.environ["WATSONX_BASE_URL"]="https://eu-de.ml.cloud.ibm.com" 

In [15]:
from termcolor import cprint

from llama_stack.distribution.library_client import LlamaStackAsLibraryClient
from llama_stack_client.types.toolgroup_register_params import McpEndpoint
from llama_stack_client import Agent, AgentEventLogger

### Prompt for agent

In [ ]:
invoice_agent_instructions = """You are a helpful invoice management assistant that can access invoice data.

CORE RESPONSIBILITIES:
1. Access invoice information using the invoice_data tool
2. Provide clear and accurate information about invoices

DATA INTEGRITY GUIDELINES:
1. Always verify data exists before making decisions or taking actions
2. If the invoice_data tool indicates no information is available, accept this response without making assumptions
3. Strictly don't make any assumptions if the tool indicates no information is available
4. For any data-driven decisions:
   - Always verify the data exists before taking actions
   - Never proceed with actions based on assumptions or incomplete data
   - If data is missing or unclear, return an answer explaining that you cannot proceed without the required information

COMMUNICATION STANDARDS:
1. Always be professional, clear, and descriptive
2. When providing invoice information, highlight key details (dates, amounts, status)

PRIVACY & SECURITY:
1. Be cautious with personally identifiable information

Follow these guidelines carefully to provide accurate, helpful invoice management assistance.
"""

### Initialize the LlamaStack client

In [ ]:
client = LlamaStackAsLibraryClient(os.getenv("INFERENCE_PROVIDER"))
client.initialize()

In [19]:
try:
    client.toolgroups.unregister(toolgroup_id="mcp::watsonx_data")
except Exception as e:
    print(e)

### Registering governed data tools from watsonx.data MCP server

In [ ]:
client.toolgroups.register(
    toolgroup_id="mcp::watsonx_data",
    provider_id="model-context-protocol",
    mcp_endpoint=McpEndpoint(uri=os.getenv("MCP_ENDPOINT")),
)

cprint("\n\n MCP Tools:", "blue")
tools = client.tools.list(toolgroup_id="mcp::watsonx_data")
for tool in tools:
    cprint(f"TOOL NAME : {tool.identifier}", "white")
    cprint(f"Description: {tool.description}", "yellow")
    cprint(f"Parameters: {tool.parameters}\n", "cyan")
    print()

### Creating an Invoice agent with governed data tools from watsonx.data MCP server and Local Notification Tools

In [25]:
invoice_agent = Agent(
    client, 
    model="meta-llama/Llama-3.3-70B-Instruct",
    instructions=invoice_agent_instructions,
    tools=["mcp::watsonx_data"],
    sampling_params={
        "max_tokens": 4095
    }
)


### Invoice Agent Execution

In [ ]:
# Create an Agent session
session_id = invoice_agent.create_session("session-alpha")

# Sample user input
user_input = '''Find the total number of invoices in system'''

# Start Agent
cprint("\n\n Invoice Management Agent started.", "blue")
cprint(f"\nTask : {user_input}")
try:
    messages = [{
        "role": "user",
        "content": user_input
    }]

    response = invoice_agent.create_turn(
        messages=messages,
        session_id=session_id,
    )
    
    print()
    for log in AgentEventLogger().log(response):
        log.print()
        
except Exception as e:
    cprint(f"\nError: {str(e)}", "red")